In [1]:
import simpy
import random
import statistics

Food delivery has become the norm in India. Ever since the pandemic started, more people are ordering from outside to avoid going out. Some restaurants have also launched their own delivery services, but it's hard to market your delivery service if you aren't a popular and big enough restaurant. So there are platforms like Zomato, Swiggy etc. which serve as a common platform allowing for customers to browse through and order from an extensive list of food outlets. These platforms also hire delivery executives to make sure people's food is delivered to their doorstep. One interesting problem in such scenario would be to find the optimal number of executives to hire, which would reduce the wait times experienced by customers during assigning of an executive to an order, whilst being within the budget. We will try to simulate this problem using the python library **simpy**, which lets us simulate real life processes like the one I just described.

**Goal**:: Find the optimal number of executives and staff which gives an average wait time of less than 15 minutes. This script can be used by the delivery service CEO to act hire these many people!

In [2]:
wait_times = []

In [3]:
# step 1. user requests an order from a restrau
# 2. restrau accepts/rejects the order based on if items are avilable
# 3. after some time, a deli exec gets assigned
# 4. once the order is prepared, deli exec picks up the order and starts out for the user's address
#5. deli exec delivers the order
# right now, we concerned with 1-3 since 4 and 5 can vary highly depending on user address and restrau location

Having ordered from both Zomato and Swiggy countless times, I can rely on my personal experience to estimate the average time it takes for a delivery person to get assigned. 

In [4]:
class DeliveryService(object):
    def __init__(self, env, num_delivery_execs):
        self.env = env
        # num_delivery_execs = maximum number of execs available
        self.delivery_execs = simpy.Resource(env, num_delivery_execs)
        # in future, we can add self.restrau
    
    def assign_exec(self, restr):
        # Times out after a random 
        # an order must be there to get an exec assigned to!
        yield self.env.timeout(random.randint(3,5))
    
    def reach_restaurant(self, restr):
        yield self.env.timeout(random.randint(10,15))
    
    def deliver_order(self, restr, delivery_exec):
        yield self.env.timeout(random.randint(15,20))
    
class Restaurant(object):
    def __init__(self, env, num_employees):
        self.env = env
        self.employees = simpy.Resource(env, num_employees)
    
    def prepare_order(self):
        yield self.env.timeout(random.randint(20,25))
        
    def handover_to_exec(self, deliveryExec):
        # assuming exec has already reched the restaurant
        yield self.env.timeout(random.randint(1,2))

def place_order(env, deliveryService, restaurant):
    # Order arrives
    order_time = env.now
    
    # Request for a delivery exec
    with deliveryService.delivery_execs.request() as exec_request:
        #wait for order to be accepted
        yield exec_request
        #wait for exec being assigned
        yield env.process(deliveryService.assign_exec(restaurant))
        #using 'with' frees up the resources at the end of this logic
        #restaurant prepares the order
        with restaurant.employees.request() as staff_request:
            yield staff_request
            yield env.process(restaurant.prepare_order())
            yield env.process(deliveryService.reach_restaurant(restaurant))
            yield env.process(restaurant.handover_to_exec(exec_request))
        yield env.process(deliveryService.deliver_order(exec_request))
    
    wait_times.append(env.now - order_time)

In [5]:
def run_simulation(env, num_delivery_execs, restr_num_employees, restr_num_orders):
    restaurants = []
    deliveryService = DeliveryService(env, num_delivery_execs)
    
    # for each restaurant registered with the service, initialize a REstaurant object
    for restr in restr_num_employees:
        restaurants.append(Restaurant(env, restr_num_employees.get(restr)))
    
    # orders to start with
    for restr in restaurants:
        env.process(place_order(env, deliveryService, restr))
        
    # one extra order to a random restaurant every minute
    while True:
        ind = random.randint(0, len(restaurants)-1)
        restr = restaurants[ind]
        yield env.timeout(1) # based on data/experience/information
        env.process(place_order(env, deliveryService, restr)) # include order as a number

In [6]:
def get_average_wait_time(wait_times):
    average_time = statistics.mean(wait_times)
    #Pretty printing the results
    minutes, fraction_minutes = divmod(average_time, 1)
    sec = fraction_minutes * 60
    return round(minutes), round(sec)

In [7]:
# Get the count inputs from user. Ex: the delivery service CEO!
def get_count_inputs():
    restr_num_orders = {}
    restr_num_employees = {}
    try:
        time = int(input('Input number of minutes to run the simulation for: '))
        num_delivery_execs = int(input('Input number of delivery executives you have: '))
        number_of_restaurants = int(input('Input number of restaurants registered: '))
        while(number_of_restaurants):
            restr_name = input('Input name of restaurant: ')
            restr_orders = int(input('Input number of orders waiting at this restaurant at the beginning: '))
            restr_employees = int(input('Input number of employees working here for preparing orders: '))
            restr_num_orders[restr_name] = restr_orders
            restr_num_employees[restr_name] = restr_employees
            number_of_restaurants -= 1
    except ValueError as e:
        get_count_inputs()
    return (num_delivery_execs, restr_num_employees, restr_num_orders, time)

In [8]:
def main():
    random.seed(100)
    num_delivery_execs, restr_num_employees, restr_num_orders, time = get_count_inputs()
    
    # Run the simulation
    env = simpy.Environment()
    env.process(run_simulation(env, num_delivery_execs, restr_num_employees, restr_num_orders))
    # Run for these many minutes
    env.run(until=time)
    
    # View the results
    mins, secs = get_average_wait_time(wait_times)
    print(f'Ran simulation; Average wait time is: {mins} minutes {secs} seconds')
    
if __name__ == '__main__':
    main()

Input number of minutes to run the simulation for: 5
Input number of delivery executives you have: 2
Input number of restaurants registered: 1
Input name of restaurant: wera
Input number of orders waiting at this restaurant at the beginning: 1
Input number of employees working here for preparing orders: 2


StatisticsError: mean requires at least one data point